<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_absolute_reality_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_absolute_reality_ai.py

Pipeline:
1. Synthetic “reality substrate” dataset (6 inputs → 3 targets)
2. Float64 data generation → Float32 normalization
3. Enhanced MLP with LayerNorm, Dropout & ReLU
4. Physics-informed residual enforcing toy “fabrication laws”
5. MC-Dropout for epistemic uncertainty
6. Training loop: AdamW, ReduceLROnPlateau, grad clipping, NaN checks, early stopping
7. Checkpointing & safe model reload
8. Visualizations: loss curves, scatter plots, uncertainty map
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Reality Substrate Dataset
# ------------------------------------------------------------------------------
class RealityDataset(Dataset):
    def __init__(self, n_samples=5000, seed=123):
        np.random.seed(seed)
        # Inputs: pre-entropy (PE), causality seeds (CS1–CS4), coherence metric (CM)
        PE  = np.random.uniform(0.0, 1.0, (n_samples,1))
        CS1 = np.random.uniform(0.0, 5.0, (n_samples,1))
        CS2 = np.random.uniform(0.0, 5.0, (n_samples,1))
        CS3 = np.random.uniform(0.0, 5.0, (n_samples,1))
        CS4 = np.random.uniform(0.0, 5.0, (n_samples,1))
        CM  = np.random.uniform(0.1, 2.0, (n_samples,1))
        X_raw = np.hstack([PE, CS1, CS2, CS3, CS4, CM]).astype(np.float64)

        # Toy fabrication laws → targets
        eps = 1e-6
        # 1. Reality Emergence (RE)
        RE = PE * np.log1p(CS1 + CS2 + eps) * CM
        # 2. Substrate Coherence (SC)
        SC = (CS1 * CS2 * CS3 * CS4)**0.25 / (1 + PE)
        # 3. Trans-dimensional Balance (TB)
        TB = np.sin(PE * np.pi) * CM

        Y_raw = np.hstack([RE, SC, TB]).astype(np.float64)
        Y_raw += 0.02 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Compute stats
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0) + 1e-8
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0) + 1e-8

        # Normalize to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. Enhanced AbsoluteRealityAI Definition
# ------------------------------------------------------------------------------
class AbsoluteRealityAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dims=(64, 64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, d = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(d, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            d = h
        layers.append(nn.Linear(d, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-informed Residual Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_denorm = X * stats['X_std'] + stats['X_mean']
    PE, CS1, CS2, CS3, CS4, CM = X_denorm.t()
    eps = 1e-6

    RE_t = PE * torch.log1p(CS1 + CS2 + eps) * CM
    SC_t = (CS1 * CS2 * CS3 * CS4)**0.25 / (1 + PE)
    TB_t = torch.sin(PE * np.pi) * CM

    Y_true = torch.stack([RE_t, SC_t, TB_t], dim=1)
    Y_norm = (Y_true - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Y_norm)

# ------------------------------------------------------------------------------
# 4. Combined Loss
# ------------------------------------------------------------------------------
def total_loss(pred, true, X, stats, lam=1.0):
    mse_loss = nn.MSELoss()(pred, true)
    phys_loss = physics_residual(pred, X, stats)
    return mse_loss + lam * phys_loss, mse_loss, phys_loss

# ------------------------------------------------------------------------------
# 5. MC-Dropout Uncertainty
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    all_preds = torch.stack(preds)
    return all_preds.mean(0), all_preds.std(0)

# ------------------------------------------------------------------------------
# 6. Training Loop & Checkpointing
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-4, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=5)
    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, epochs+1):
        # -- Train --
        model.train()
        train_loss = 0
        for Xb, Yb in train_loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats, lam)
            if torch.isnan(loss):
                print(f"NaN at epoch {epoch}, abort.")
                return history
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            train_loss += loss.item() * Xb.size(0)
        train_loss /= len(train_loader.dataset)

        # -- Validate --
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for Xv, Yv in val_loader:
                Xv, Yv = Xv.to(device), Yv.to(device)
                pred = model(Xv)
                l, _, _ = total_loss(pred, Yv, Xv, stats, lam)
                val_loss += l.item() * Xv.size(0)
        val_loss /= len(val_loader.dataset)

        sched.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Checkpoint
        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_absolute_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    # Reload best
    if os.path.exists("best_absolute_ai.pth"):
        model.load_state_dict(torch.load("best_absolute_ai.pth", map_location=device))
    return history

# ------------------------------------------------------------------------------
# 7. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
    plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    mn, mx = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
    plt.plot([mn, mx],[mn, mx],'r--')
    plt.title(name); plt.show()

def plot_uncertainty(model, stats, device):
    G = 100
    PE = np.linspace(0,1,G, dtype=np.float32)
    CM = np.linspace(0.1,2.0,G, dtype=np.float32)
    grid = np.array(np.meshgrid(PE, CM)).T.reshape(-1,2)
    Xg = np.zeros((G*G,6), dtype=np.float32)
    # fill PE, CM; keep CS1–4 at mean
    Xg[:,0] = grid[:,0]; Xg[:,5] = grid[:,1]
    Xg[:,1:5] = stats['X_mean'][1:5].cpu().numpy()

    Xn = (torch.from_numpy(Xg).to(device) - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=40)
    U = std[:,0].cpu().numpy().reshape(G,G)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(PE, CM, U, shading='auto', cmap='viridis')
    plt.colorbar(label="Std(RE)")
    plt.xlabel("Pre-entropy"); plt.ylabel("Coherence")
    plt.title("Uncertainty: Reality Emergence")
    plt.show()

# ------------------------------------------------------------------------------
# 8. Main
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset & stats
    ds = RealityDataset(n_samples=6000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device, dtype=torch.float32),
        'X_std' : torch.tensor(ds.X_std,  device=device, dtype=torch.float32),
        'Y_mean': torch.tensor(ds.Y_mean, device=device, dtype=torch.float32),
        'Y_std' : torch.tensor(ds.Y_std,  device=device, dtype=torch.float32),
    }

    # Split & loaders
    n_val = int(0.2 * len(ds))
    train_ds, val_ds = random_split(ds, [len(ds)-n_val, n_val])
    tr_ld = DataLoader(train_ds, batch_size=128, shuffle=True)
    va_ld = DataLoader(val_ds,   batch_size=256, shuffle=False)

    # Model, Train, Visualize
    model = AbsoluteRealityAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    # True vs Predicted
    X_all = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        Yp_n = model(X_all).cpu().numpy()
    Y_true = ds.Y * ds.Y_std + ds.Y_mean
    Y_pred  = Yp_n * ds.Y_std + ds.Y_mean
    names = ["Reality Emergence","Substrate Coherence","Trans-dimensional Balance"]
    for i, nm in enumerate(names):
        plot_scatter(Y_true[:,i], Y_pred[:,i], nm)

    plot_uncertainty(model, stats, device)